In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
train = pd.read_csv('train_clean.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,total_fam,prefix
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,Mr.
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,Mrs.
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Miss.
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,Mrs.
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,Mr.


In [4]:
train['Sex'] = train['Sex'].astype('category')
train['Pclass'] = train['Pclass'].astype('category')
train['Embarked'] = train['Embarked'].astype('category')
train['prefix'] = train['prefix'].astype('category')

In [5]:
train = pd.get_dummies(train, drop_first=True, columns=['Sex','Pclass','Embarked','prefix'])

In [6]:
train.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,total_fam,...,prefix_Master.,prefix_Miss.,prefix_Mlle.,prefix_Mme.,prefix_Mr.,prefix_Mrs.,prefix_Ms.,prefix_Rev.,prefix_Sir.,prefix_the
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,2,...,0,0,0,0,1,0,0,0,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,2,...,0,0,0,0,0,1,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,...,0,1,0,0,0,0,0,0,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,2,...,0,0,0,0,0,1,0,0,0,0
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,1,...,0,0,0,0,1,0,0,0,0,0


In [7]:
train.drop(['PassengerId','Name','Ticket','Cabin'], 1, inplace=True)

In [35]:
list(train.columns)

['Survived',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'total_fam',
 'Sex_male',
 'Pclass_2',
 'Pclass_3',
 'Embarked_Q',
 'Embarked_S',
 'prefix_Col.',
 'prefix_Don.',
 'prefix_Dr.',
 'prefix_Jonkheer.',
 'prefix_Lady.',
 'prefix_Major.',
 'prefix_Master.',
 'prefix_Miss.',
 'prefix_Mlle.',
 'prefix_Mme.',
 'prefix_Mr.',
 'prefix_Mrs.',
 'prefix_Ms.',
 'prefix_Rev.',
 'prefix_Sir.',
 'prefix_the']

In [9]:
X = train.drop('Survived', 1)
y = train['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [10]:
c_space = np.linspace(0.0001, 10, 15)
param_grid1 = {'C': c_space} 

In [11]:
svc = SVC(kernel='poly', degree=2)

svc_rs = GridSearchCV(svc, param_grid1, cv=3,scoring='roc_auc')

svc_rs.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-04,   7.14379e-01,   1.42866e+00,   2.14294e+00,
         2.85721e+00,   3.57149e+00,   4.28577e+00,   5.00005e+00,
         5.71433e+00,   6.42861e+00,   7.14289e+00,   7.85716e+00,
         8.57144e+00,   9.28572e+00,   1.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [12]:
pred2 = svc_rs.predict(X_test)
svc_rs.score(X_test, y_test)

0.84736164736164743

In [13]:
print(classification_report(y_test, pred2))

             precision    recall  f1-score   support

          0       0.84      0.88      0.86       105
          1       0.81      0.76      0.78        74

avg / total       0.83      0.83      0.83       179



In [14]:
confusion_matrix(y_test, pred2)

array([[92, 13],
       [18, 56]], dtype=int64)

In [15]:
roc_auc_score(y_test, pred2)

0.81647361647361649

In [27]:
test = pd.read_csv('test_clean.csv')

In [28]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,total_fam,prefix
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1,Mr.
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,2,Mrs.
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1,Miss.
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1,Mrs.
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,3,Mr.


In [29]:
ids = test['PassengerId']

In [30]:
test['Sex'] = test['Sex'].astype('category')
test['Pclass'] = test['Pclass'].astype('category')
test['Embarked'] = test['Embarked'].astype('category')
test['prefix'] = test['prefix'].astype('category')

In [31]:
test = pd.get_dummies(test, drop_first=True, columns=['Sex','Pclass','Embarked','prefix'])

In [32]:
test.drop(['PassengerId','Name','Ticket','Cabin'], 1, inplace=True)

In [46]:
test['prefix_Col.'] = 0
test['prefix_Don.'] = 0
test['prefix_Jonkheer.'] = 0
test['prefix_Lady.'] = 0
test['prefix_Major.'] = 0
test['prefix_Mlle.'] = 0
test['prefix_Don.'] = 0
test['prefix_Don.'] = 0
test['prefix_Don.'] = 0

In [47]:
list(test.columns)

['Age',
 'SibSp',
 'Parch',
 'Fare',
 'total_fam',
 'Sex_male',
 'Pclass_2',
 'Pclass_3',
 'Embarked_Q',
 'Embarked_S',
 'prefix_Dr.',
 'prefix_Master.',
 'prefix_Miss.',
 'prefix_Mme.',
 'prefix_Mr.',
 'prefix_Mrs.',
 'prefix_Rev.',
 'prefix_Col.',
 'prefix_Don.',
 'prefix_Jonkheer.',
 'prefix_Lady.',
 'prefix_Major.',
 'prefix_Mlle.',
 'prefix_Sir.',
 'prefix_the.',
 'prefix_Ms.']

In [48]:
pred = svc_rs.predict(test)

In [49]:
prob_dict = {'PassengerId':ids, 'Survived':pred}
submission = pd.DataFrame(prob_dict)

In [50]:
submission.to_csv('basic_svc_submission.csv', index=False)

In [51]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [52]:
max_depth = np.linspace(3,15)
param_grid2 = {'max_depth': max_depth, 'min_samples_split':[5,10,15], 'min_samples_leaf':[5,10,15], 'class_weight':[{0:.35,1:.65},{0:.4,1:.6},{0:.37,1:.63}]} 

In [54]:
rf = RandomForestClassifier(n_estimators=1000)

rf_rs = RandomizedSearchCV(rf, param_grid2, n_iter=25, cv=3, scoring='roc_auc')

rf_rs.fit(X_train, y_train)

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=25, n_jobs=1,
          param_distributions={'max_depth': array([  3.     ,   3.2449 ,   3.4898 ,   3.73469,   3.97959,   4.22449,
         4.46939,   4.71429,   4.95918,   5.20408,   5.44898,   5.69388,
         5.93878,   6.18367,   6.42857,   6.67347,   6.91837,   7.16327,
         7.40816,   7.65306,   7.89796,   8.142...ples_leaf': [5, 10, 15], 'class_weight': [{0: 0.35, 1: 0.65}, {0: 0.4, 1: 0.6}, {0: 0.37, 1: 0.63}]},
          pre_dispatch='2*n_jobs', ra

In [55]:
pred2 = rf_rs.predict(X_test)
rf_rs.score(X_test, y_test)

0.9027027027027027

In [56]:
print(classification_report(y_test, pred2))

             precision    recall  f1-score   support

          0       0.86      0.85      0.86       105
          1       0.79      0.81      0.80        74

avg / total       0.83      0.83      0.83       179



In [57]:
confusion_matrix(y_test, pred2)

array([[89, 16],
       [14, 60]], dtype=int64)

In [58]:
roc_auc_score(y_test, pred2)

0.82921492921492923

In [59]:
pred4 = rf_rs.predict(test)

In [60]:
prob_dict = {'PassengerId':ids, 'Survived':pred4}
submission = pd.DataFrame(prob_dict)

In [61]:
submission.to_csv('rf_submission.csv', index=False)